# MNIST Classification

🎯 <b><u>Exercise objectives</u></b>
- Understand the *MNIST* dataset 
- Design your first **Convolutional Neural Network** (*CNN*) and answer questions such as:
    - what are *Convolutional Layers* ? 
    - how many *parameters* are involved in such a layer ?
- Train this CNN on images

🚀 <b><u>Let's get started !</u></b>

Imagine that we are  back in time into the 90's.
You work at a *Post Office* and you have to deal with an enormous amount of letters on a daily basis. How could you automate the process of reading the ZIP Codes, which are a combination of 5 handwritten digits ? 

This task, called the **Handwriting Recognition**, used to be a very complex problem back in those days. It was solved by *Bell Labs* (among others) where one of the Deep Learning gurus, *Yann Le Cun* used to work.

From [Wikipedia](https://en.wikipedia.org/wiki/Handwriting_recognition):

> Handwriting recognition (HWR), also known as Handwritten Text Recognition (HTR), is the ability of a computer to receive and interpret intelligible handwritten input from sources such as paper documents, photographs, touch-screens and other devices

![Number recognition](recognition.gif)

*Nota Bene: The animation above is just here to help you visualize what happens with the different images: <br/> $\rightarrow$ For each image, the CNN predicts what digit is written. The inputs are the different digits and not one animation/video!*

🤔 <b><u>How does this CNN work ?</u></b>

- *Inputs*: Images (_each image shows a handwritten digit_)
- *Target*: For each image, you want your CNN model to predict the correct digit (between 0 and 9)
    - It is a **multi-class classification** task (more precisely a 10-class classification task since there are 10 different digits).

🔢 To improve the capacity of the Convolutional Neural Network to read these numbers, we need to feed it with many images representing handwritten digits. This is why the 📚 [**MNIST dataset**] 📚(http://yann.lecun.com/exdb/mnist/) *(Mixed National Institute of Standards and Technology)* was created.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## (1) The `MNIST` Dataset

📚 Tensorflow/Keras offers multiple [**datasets**](https://www.tensorflow.org/api_docs/python/tf/keras/datasets) to play with:
- *Vectors*: `boston_housing` (regression)
- *Images* : `mnist`, `fashion_mnist`, `cifar10`, `cifar100` (classification)
- *Texts*: `imbd`, `reuters` (classification/sentiment analysis)


💾 You can **load the MNIST dataset** with the following commands:

In [21]:
from tensorflow.keras import datasets

(X_train, y_train), (X_test, y_test) = datasets.mnist.load_data(path="mnist.npz")

### (1.1) Exploring the dataset

❓ **Question: Let's have look at some handwritten digits of this MNIST dataset.** ❓

🖨 Print some images from the *train set*.

<details>
    <summary><i>Hints</i></summary>

💡*Hint*: use the `imshow` function from `matplotlib` with `cmap = "gray"`

🤨 Note: if you don't specify this *cmap* argument, the weirdly displayed colors are just some arrangements done Matplotlib...
    
</details>

In [17]:
import pandas as pd

### (1.2) Image Preprocessing

❗️ **Neural Networks converge faster when the input data is somehow normalized** ❗️

👩🏻‍🏫 How do we proceed for Convolutional Neural Networks ?
* The `RBG` intensities are coded between 0 and 255. 
* We can simply divide the input data by the maximal value 255 to have all the pixels' intensities between 0 and 1 😉

❓ **Question: As a first preprocessing step, please normalize your data.** ❓

Don't forget to do it both on your train data and your test data.

(*NB: you can also center your data, by substracting 0.5 from all the values, but it is not mandatory*)

In [22]:
X_train = X_train / 255. - 0.5
X_test = X_test / 255. - 0.5

### (1.3) Inputs' dimensionality

❓ **Question: What is the shape of your images** ❓

In [26]:
X_train.shape

(60000, 28, 28)

👆 You see that you have 60.000 training images, all of size $(28, 28)$. However...

❗️  **`Convolutional Neural Network models need to be fed with images whose last dimension is the number of channels`** ❗️

This last dimension is clearly missing here... Can you guess the reason why ?

<details>
    <summary><i>Answer<i></summary>
        
* All these $60000$ $ 28 \times 28 $ pictures are black-and-white. $ \implies $ Each pixel lives on a spectrum from full black (0) to full white (1)
        
* Theoretically, you don't need to know the number of channels unlike colored pictures using for example:
    - the RGB system with 3 channels (<b><span style="color:red">Red</span> <span style="color:green">Green</span> <span style="color:blue">Blue</span></b>)
    - the CYMK system  with 4 channels (<b><span style="color:cyan">Cyan</span> <span style="color:magenta">Magenta</span> <span style="color:yellow">Yellow</span> <span style="color:black">Black</span> </b>)
        
        
</details>        

❓ **Question: expanding dimensions** ❓

* Use the **expand_dims** to add one dimension at the end of the training data and test data.

* Then, print the shapes of `X_train` and `X_test` which should respectively be equal to  $(60000, 28, 28, 1)$ and $(10000, 28, 28, 1)$.

In [27]:
from tensorflow.keras.backend import expand_dims

In [28]:
X_train = expand_dims(X_train)

2022-02-16 11:17:09.917163: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [31]:
X_test = expand_dims(X_test)

### (1.4) Label encoding

A last thing to do to prepare your data is to convert your labels to "*one-hot encode*" the categories.

❓ **Question: encoding the labels** ❓ 

* Use **`to_categorical`** to transform your labels. 
* Store the results into two variables that you can call **`y_train_cat`** and **`y_test_cat`**.

In [32]:
from tensorflow.keras.utils import to_categorical

y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

The data is now ready to be used. ✅

## (2) The Convolutional Neural Network

### (2.1) Architecture and compilation of a CNN


❓ **Question: CNN Architecture and compilation** ❓

Now, let's build a <u>Convolutional Neural Network</u> that has: 


- a `Conv2D` layer with 8 filters, each of size $(4, 4)$, an input shape suitable for your task, the `relu` activation function, and `padding='same'`
- a `MaxPool2D` layer with a `pool_size` equal to $ (2, 2) $
- a second `Conv2D` layer with 16 filters, each of size $ (3, 3) $, and the `relu` activation function
- a second `MaxPool2D` layer with a `pool_size` equal to $ (2, 2) $


- a `Flatten` layer
- a first `Dense` layer with 10 neurons and the `relu` activation function
- a last (predictive) layer that is suited for your task

In the function that initializes this model, do not forget to include the <u>compilation of the model</u>, which:
* optimizes the `categorical_crossentropy` 
* with the `adam` optimizer
* and the `accuracy` as the metrics

(*NB: you could add more classification metrics if you want but the dataset is well balanced!*)

In [34]:
from tensorflow.keras import layers
from tensorflow.keras import models


def initialize_model():

    model = models.Sequential()

    ### First Convolution & MaxPooling
    model.add(layers.Conv2D(8, kernel_size=(4,4), input_shape=(28,28,1), activation ='relu'))
    model.add(layers.MaxPool2D(pool_size=(2,2)))
    
    ### Second Convolution & MaxPooling
    model.add(layers.Conv2D(8, kernel_size=(3,3), activation ='relu'))
    model.add(layers.MaxPool2D(pool_size=(2,2))) 
    
    ### Flattening
    model.add(layers.Flatten())
    
    ### One Fully Connected layer - "Fully Connected" is equivalent to saying "Dense"
    model.add(layers.Dense(10, activation='relu'))
    
    ### Last layer - Classification Layer with 10 outputs corresponding to 10 digits
    model.add(layers.Dense(10, activation='softmax'))
    
    ### Model compilation
    model.compile( optimizer='adam',
                 metrics='accuracy',
                 loss = 'categorical_crossentropy')
    
    return model

❓ **Question: number of trainable parameters in a convoluational layer** ❓ 

How many trainable parameters are there in your model?
1. Compute them with ***model.summary( )*** first
2. Recompute them manually to make sure you properly understood ***what influences the number of weights in a CNN***.

In [45]:
 8 * 8 * (3 * 3)  + 8 # + 2010 + 110

584

In [40]:
initialize_model().summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 25, 25, 8)         136       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 12, 12, 8)         0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 10, 10, 8)         584       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 5, 5, 8)           0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 200)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 10)                2010      
_________________________________________________________________
dense_11 (Dense)             (None, 10)               

### (2.2) Training a CNN

❓ **Question: training a CNN** ❓ 

Initialize your model and fit it on the train data. 
- Do not forget to use a **Validation Set/Split** and an **Early Stopping criterion**. 
- Limit yourself to 10 epochs max in this challenge, just to save some precious time for the more advanced challenges... !

In [49]:
from sklearn.model_selection import train_test_split

In [51]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience = 3, restore_best_weights=True)

model = initialize_model()
model.fit(X_train, y_train_cat, validation_split=0.5, callbacks=[es], epochs = 10)

Epoch 1/10
938/938 [==============================] - 5s 5ms/step - loss: 0.5415 - accuracy: 0.8396 - val_loss: 0.2100 - val_accuracy: 0.9401
Epoch 2/10
938/938 [==============================] - 5s 5ms/step - loss: 0.1635 - accuracy: 0.9512 - val_loss: 0.1599 - val_accuracy: 0.9517
Epoch 3/10
938/938 [==============================] - 4s 4ms/step - loss: 0.1241 - accuracy: 0.9632 - val_loss: 0.1215 - val_accuracy: 0.9635
Epoch 4/10
938/938 [==============================] - 4s 5ms/step - loss: 0.1033 - accuracy: 0.9685 - val_loss: 0.1058 - val_accuracy: 0.9662
Epoch 5/10
938/938 [==============================] - 4s 5ms/step - loss: 0.0893 - accuracy: 0.9726 - val_loss: 0.1072 - val_accuracy: 0.9660
Epoch 6/10
938/938 [==============================] - 4s 4ms/step - loss: 0.0829 - accuracy: 0.9748 - val_loss: 0.1010 - val_accuracy: 0.9698
Epoch 7/10
938/938 [==============================] - 4s 4ms/step - loss: 0.0757 - accuracy: 0.9775 - val_loss: 0.0833 - val_accuracy: 0.9742
Epoch 

❓ **Question: How many iterations does the CNN perform per epoch** ❓

_Note: it has nothing to do with the fact that this is a CNN. This is related to the concept of forward/backward propagation already covered during the previous lecture on optimizers, fitting and losses 😉_

> YOUR ANSWER HERE

<details>
    <summary><i>Answer:</i></summary>

With `verbose = 1` when fitting your model, you have access to crucial information about your training procedure.
    
Remember that we've just trained our CNN model on $60000$ images from the *train images*.

If the chosen batch size is 32: 

* For each epoch, we have $ \large \lceil \frac{60000}{32} \rceil = 1875$ minibatches <br/>
* The _validation_split_ is equal to $0.3$ - which means that within one single epoch, there are:
    * $ \lceil 1875 \times (1 - 0.3) \rceil = \lceil 1312.5 \rceil = 1313$ batches are used to compute the `train_loss` 
    * $ 1875 - 1312 = 562 $  batches are used to compute the `val_loss`
    * **The parameters are updated 1313 times per epoch** as there are 1313 forward/backward propagations per epoch !!!


👉 With so many updates of the weights within one epoch, you can understand why this CNN model converges even with a limited number of epochs.

</details>    


### (2.3) Evaluating its performance

❓ **Question: Evaluating your CNN** ❓ 

What is your **`accuracy on the test set?`**

In [52]:
model.evaluate(X_test, y_test_cat)

313/313 [==============================] - 0s 1ms/step - loss: 0.0665 - accuracy: 0.9793


[0.06654331833124161, 0.9793000221252441]

🎉 You should already be impressed by your CNN skills! Reaching over 95% accuracy!

🔥 You solved what was a very hard problem 30 years ago with your own CNN.

🏁 **Congratulations!**

💾 Don't forget to `git add/commit/push` your notebook...

🚀 ... and move on to the next challenge !